In [1]:
%notebook logs/tugda_da.ipynb

In [2]:
%notebook logs/tugda_da.ipynb

In [3]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output

In [4]:
import pandas as pd
import numpy as np
import random
from itertools import cycle

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Function

In [6]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

In [7]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output

In [8]:
y=torch.zeros(3,2)

In [9]:
y

In [10]:
y=torch.rand(5,5,4)

In [11]:
y=torch.rand(5,5,4)
print(y)

In [12]:
preds_mean = torch.mean(y, axis=2)
preds_var = torch.var(y, axis=2)
total_unc = torch.mean(preds_var, axis=0)

In [13]:
print(preds_mean)

In [14]:
print(preds_mean)
print(preds.shape)

In [15]:
print(preds_mean)
print(preds_mean.shape)

In [16]:
print(preds_var)
print(preds_var.shape)

In [17]:
print(total_unc)
print(total_unc.shape)

In [18]:
y=torch.rand(5,6,4)
print(y)

In [19]:
preds_mean = torch.mean(y, axis=2)
preds_var = torch.var(y, axis=2)
total_unc = torch.mean(preds_var, axis=0)

In [20]:
print(preds_mean)
print(preds_mean.shape)

In [21]:
print(preds_var)
print(preds_var.shape)

In [22]:
print(total_unc)
print(total_unc.shape)

In [23]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [24]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [25]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 50,
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 50}

In [26]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [27]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.x_target_unl))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)

        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(self.train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0), device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0), device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

                return loss, m_loss, d_loss

In [28]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [29]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 5,
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 50}

In [30]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 5,
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}

In [31]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)

        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(self.train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0), device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0), device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [32]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [33]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [34]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 5, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [35]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [36]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [37]:
model.train()

In [38]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0), device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0), device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [39]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [40]:
model.train()

In [41]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [42]:
model.train()

In [43]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

In [44]:
device = 'cpu' if torch.backends.mps.is_available() else 'cpu'
print(device)

In [45]:
import pandas as pd
import numpy as np
import random
from itertools import cycle

In [46]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Function

In [47]:
device = 'cpu' if torch.backends.mps.is_available() else 'cpu'
print(device)

In [48]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [49]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0), device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0), device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [50]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [51]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [52]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 5, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [53]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [54]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [55]:
model.train()

In [56]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [57]:
model.train()

In [58]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [59]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [60]:
model.train()

In [61]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0))#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0))#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [62]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)

In [63]:
model.train()

In [64]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [65]:
model.train()

In [66]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [67]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [68]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                print({'total_loss': loss})#, 'source_loss': task_loss, 'disc_loss': d_loss})

        return None #loss, m_loss, d_loss

In [69]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [70]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [71]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 5, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [72]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [73]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [74]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 50, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [75]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [76]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [77]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        train_dataloader=self.train_dataloader()
        for epoch in range(self.n_epoch):
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

            print({'total_loss': loss})#, 'source_loss': task_loss, 'disc_loss': d_loss})
        print("hello")
        return None #loss, m_loss, d_loss

In [78]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [79]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [80]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 50, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [81]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [82]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [83]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diag or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        
        for epoch in range(self.n_epoch):
            train_dataloader=self.train_dataloader()
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

            print({'total_loss': loss})#, 'source_loss': task_loss, 'disc_loss': d_loss})
        return None #loss, m_loss, d_loss

In [84]:
#cell-line dataset;
gdsc_dataset = pd.read_csv('data/GDSCDA_fpkm_AUC_all_drugs.zip', index_col=0)
#gene set range
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

In [85]:
#pdx novartis dataset;
pdx_dataset = pd.read_csv('data/PDX_MTL_DA.csv', index_col=0)
drugs_pdx = pdx_dataset.columns[1780:]

In [86]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 10, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [87]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 10, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [88]:
gene_list = gdsc_dataset.columns[0:1780]
drug_list = gdsc_dataset.columns[1780:]

X_train = gdsc_dataset[gene_list].values
y_train = gdsc_dataset[drug_list].values

X_test = pdx_dataset[gene_list].values
y_test = pdx_dataset[drugs_pdx].values

In [89]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [90]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 50, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 10}#50

In [91]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [92]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 50, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 50}#50

In [93]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [94]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diagram or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        
        for epoch in range(self.n_epoch):
            train_dataloader=self.train_dataloader()
            train_loss=0
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                train_loss+=loss

            train_loss/=self.len_dataloader
            print('Epoch [{}/{}] : Loss {}'.format(epoch,self.n_epoch,train_loss))#, 'source_loss': task_loss, 'disc_loss': d_loss})
        return None #loss, m_loss, d_loss

In [95]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()

In [96]:
class Tugda_da(nn.Module):

    def __init__(self,params,train_data,test_data,y_train,y_test):
        super(Tugda_da,self).__init__()
        self.learning_rate = params['lr']
        self.batch_size = params['bs_source']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.lambda_disc = params['lambda_disc']
        self.bs_disc = params['bs_disc']
        self.n_epoch = params['n_epochs']
        self.passes = params['passes']
        self.num_tasks = params['num_tasks']

        self.train_data = train_data
        self.y_train = y_train
        self.test_data = test_data
        self.y_test = y_test

        input_dim=self.train_data.shape[1]

        feature_extractor = [nn.Linear(input_dim, params['hidden_units_1']), 
                             nn.Dropout(p=params['dropout']),
                             nn.ReLU()]
        
        self.feature_extractor = nn.Sequential(*feature_extractor) #L layer in diag

        latent_basis =  [nn.Linear(params['hidden_units_1'], params['latent_space']),
                         nn.Dropout(p=params['dropout']),
                         nn.ReLU()]
        
        self.latent_basis = nn.Sequential(*latent_basis) #Z layer in diag
        
        #task-specific weights
        self.MTL = nn.Linear(params['latent_space'], self.num_tasks) #S Layer in diag| also the prediction outputs
        
        #decoder weights
        decTTF = [nn.Linear( self.num_tasks , params['latent_space']), nn.ReLU()] # A layer in diagram or decoder task to feature layer 
        self.decTTF = nn.Sequential(*decTTF) 

        #uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
        #discriminator
        task_classifier = [nn.Linear(params["latent_space"], params["n_units_disc"]), 
                             nn.Dropout(p=params['dropout']), 
                             nn.ReLU(),
                             nn.Linear(params["n_units_disc"], 1), 
                             nn.Sigmoid()
                            ]
        self.task_classifier = nn.Sequential(*task_classifier)

        self.prepare_data()

    def forward(self, input_data, alpha):
        x = self.feature_extractor(input_data) #usually the input data is treated as x
        x = self.latent_basis(x) # here h 
        preds = self.MTL(x)
        x_hat = self.decTTF(preds) # task transfer to features # here was h_hat
        reverse_feature = ReverseLayerF.apply(x, alpha)
        task_classifier_output = self.task_classifier(reverse_feature)
        return preds, x, x_hat, task_classifier_output
    
    def prepare_data(self):
        train_dataset = TensorDataset(torch.FloatTensor(self.train_data),
                                      torch.FloatTensor(self.y_train))

        
        target_unl_dataset = TensorDataset(torch.FloatTensor(self.test_data))
        
        self.train_dataset = train_dataset
        self.target_unl_dataset = target_unl_dataset
    
    def train_dataloader(self):
        dataloader1 = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True) #in vitro data
        dataloader2 = DataLoader(self.target_unl_dataset, batch_size=self.bs_disc, shuffle=True) #in vivo data 
        self.len_dataloader = min(len(dataloader1), len(dataloader2))
        x=zip(dataloader1, cycle(dataloader2))
        return x
    
    def test_dataloader(self):
        dataloader = DataLoader(self.target_unl_dataset, batch_size=len(self.target_unl_dataset), shuffle=False)
        return dataloader
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def task_mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k]) #didnt understand this.
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss

    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        per_sample_loss = torch.zeros(labels.size(0), device=device)

        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
        
        #per class loss
        for k in range(labels.size(0)):
            per_sample_loss[k] = mse_loss(preds[k, ~torch.isnan(labels[k,:])], labels[k, ~torch.isnan(labels[k, :])])
        
        
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss,per_sample_loss
    
    def binary_classification_loss(self, preds, labels):
        bin_loss = torch.nn.BCELoss()
        return bin_loss(preds, labels)
    
    def train(self):

        optimizer = torch.optim.Adagrad([
            {'params': self.feature_extractor.parameters()},
            {'params': self.latent_basis.parameters()},
            {'params': self.MTL.parameters()},
            {'params': self.decTTF.parameters()},
            {'params': self.task_classifier.parameters()},
            {'params': self.log_vars}
        ], lr=self.learning_rate)
        
        for epoch in range(self.n_epoch):
            train_dataloader=self.train_dataloader()
            train_loss=0
            for batch_idx,data in enumerate(train_dataloader):
                x,y=data[0] #in vitro
                x_unl=data[1][0] #invivo

                p=float(batch_idx+epoch*self.len_dataloader)/self.n_epoch/self.len_dataloader
                alpha=2. / (1. + np.exp(-10 * p)) - 1

                preds_simulation=torch.zeros(y.size(0),y.size(1),self.passes)#,device=device) 
                for simulation in range(self.passes):
                    preds, h, h_hat, domain_out_source = self.forward(x, alpha)
                    preds_simulation[:,:, simulation]=preds
                
                preds_mean = torch.mean(preds_simulation, axis=2) #2D predictions after p passes
                preds_var = torch.var(preds_simulation, axis=2) #2D
                total_unc = torch.mean(preds_var, axis=0) # mu_t this is epistemic uncertainity 1D of tasks i.e y.shape[1]


                m_loss, task_loss = self.task_mse_ignore_nan(preds_mean, y)
                recon_loss= self.MSE_loss(h,h_hat)

                a = 1 + (total_unc + torch.sum(torch.abs(self.decTTF[0].weight.T),1)) #multiplier to L_bnn

                loss_weight = ( a[~torch.isnan(task_loss)] ) * task_loss[~torch.isnan(task_loss)]
                loss_weight = torch.sum(loss_weight)

                #Regularizer gamma(Z-sig(ZSA))
                l1_S = self.mu * self.MTL.weight.norm(1)
                L = self.latent_basis[0].weight.norm(2) + self.feature_extractor[0].weight.norm(2)
                l2_L = self.lambda_ * L

                #domain discriminator source
                #normal data pass
                # 0 is one class of invitro
                zeros = torch.zeros(y.size(0),1)#, device=self.device)
                d_loss_source = self.binary_classification_loss(domain_out_source, zeros)
                
                #domain discriminator target
                #unlabelled data pass
                #1 is second class which is invivo 
                preds, h, h_hat, domain_out_target = self.forward(x_unl, alpha)
                ones = torch.ones(x_unl.size(0),1)#, device=self.device) 
                d_loss_target = self.binary_classification_loss(domain_out_target, ones)
                
                d_loss = d_loss_source + d_loss_target #l_adv loss or the adversarial loss 

                #total loss
                loss = loss_weight + recon_loss + l1_S + l2_L + (self.lambda_disc *d_loss)

                loss.backward()
                optimizer.step()

                train_loss+=loss

            train_loss/=self.len_dataloader
            print('Epoch [{}/{}] : Loss {}'.format(epoch,self.n_epoch,train_loss))#, 'source_loss': task_loss, 'disc_loss': d_loss})
        return None #loss, m_loss, d_loss
    
    def test(self):

In [97]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 150, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 50}#50

In [98]:
net_params = {
 'hidden_units_1': 1500,
 'latent_space': 800,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 1,
 'lambda_': 1,
 'gamma': 0.01,
 'n_units_disc': 500,
 'n_epochs': 150, #50
 'bs_disc': 64,
 'bs_source': 300,
 'lambda_disc': 0.3,
 'num_tasks': 200,
 'passes': 20}#50

In [99]:
model = Tugda_da(net_params, X_train, X_test, y_train, y_test)
model.train()